In [0]:
!pip install torch torchtext torchvision

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
!unzip WikiQACorpus.zip

In [0]:
import torch
from torch import nn, optim
import torch.nn.functional as f
from torch.autograd import Variable
from torchtext.vocab import GloVe
from torchtext import data

In [0]:
class WikiQA_dataset():
  def __init__(self,embedding_dim,batch_size):
    self.RAW = data.RawField()
    self.TEXT = data.Field(batch_first=True,
                           lower=True)
    self.LABEL = data.Field(sequential=False,
                            unk_token=None)


    self.train, self.dev, self.test = data.TabularDataset.splits(
        path='.',
        train='WikiQA-train.tsv',
        validation='WikiQA-dev.tsv',
        test='WikiQA-test.tsv',
        format='tsv',
        fields=[('qid', self.RAW),
                ('question', self.TEXT),
                ('docid', self.RAW),
                ('doctitle', self.RAW),
                ('sentenceid',self.RAW),
                ('sentence',self.TEXT),
                ('label',self.LABEL)])

    self.TEXT.build_vocab(self.train, self.dev, self.test, \
                          vectors=GloVe(name='6B', dim=embedding_dim), \
                          unk_init=torch.zeros((1, embedding_dim)).uniform_(-0.25, 0.25))

    self.LABEL.build_vocab(self.train)

    self.train_iter, self.dev_iter, self.test_iter = \
        data.Iterator.splits((self.train, self.dev, self.test),
                             batch_sizes=[batch_size] * 3,
                             shuffle=True)


class BaselineQACNN(nn.Module):
  
  def __init__(self,emb_dim,dict_size,hidden_dim,ctx_window,decision_threshold=0.5):
    super(BaselineQACNN,self).__init__()
    self.emb = nn.Embedding(dict_size,emb_dim) # with learning, try the GloVe without learning
    self.convolution_q = nn.Conv2d(emb_dim,hidden_dim,kernel_size=(emb_dim,ctx_window)) # CARE: padding??
    self.convolution_a = nn.Conv2d(emb_dim,hidden_dim,kernel_size=(emb_dim,ctx_window))
    self.h_pool = lambda t : self.horizontal_pooling(t)
    self.crit = nn.CosineEmbeddingLoss(margin=0.5)
    self.decision_threshold = decision_threshold
    
  def forward(self,q,a):
    q_embed = self.emb(q)
    a_embed = self.emb(a)
    q_enc = F.relu(self.convolution_q(q_embed))
    a_enc = F.relu(self.convolution_a(a_embed))
    r_q = self.h_pool(q_enc)
    r_a = self.h_pool(a_enc)
    return r_q,r_a
  
  # given a matrix, does a maxpool operation on the rows
  # t.view(t.size(0),-1) = flatten into a 1D tensor each outcome for each input sample
  # passing from (n,a,b) to (n,a) dimensions
  def horizontal_pooling(t):
    return f.max_pool1d(t,t.size(2)).view(t.size(0),-1)
  
  def compute_batch_stats(self,model_output,batch):
    r_q,r_a = model_output    
    batch_loss = self.crit(r_q,r_a,batch.label.float()*2.0-1) # the loss wants -1|1 values, input values are 0|1
    sim = f.cosine_similarity(r_q,r_a)
    # compute the decision of the network
    pred = sim.clone()
    pred[sim > self.decision_threshold] = 1
    pred[sim <= self.decision_threshold] = 0
    correct_pred = (pred.squeeze() == batch.label.float()).sum().float()
    return sim,batch_loss,correct_pred
    

class BaselineQAbiLSTM(nn.Module):
  
  def __init__(self,emb_dim,dict_size,single_hidden_dim,decision_threshold=0.5):
    super(BaselineQAbiLSTM,self).__init__()
    self.emb = nn.Embedding(dict_size,emb_dim) # with learning, try the GloVe without learning
    self.bilstm_q = nn.LSTM(emb_dim,single_hidden_dim,bidirectional=True)
    self.bilstm_a = nn.LSTM(emb_dim,single_hidden_dim,bidirectional=True)
    self.h_pool = lambda t : self.horizontal_pooling(t)
    self.crit = nn.CosineEmbeddingLoss(margin=0.5)
    self.decision_threshold = decision_threshold
    
  def forward(self,q,a):
    q_embed = self.emb(q)
    a_embed = self.emb(a)
    q_enc = F.relu(self.bilstm_q(q_embed))
    a_enc = F.relu(self.bilstm_a(a_embed))
    r_q = self.h_pool(q_enc)
    r_a = self.h_pool(a_enc)
    
    return r_q,r_a
  # given a matrix, does a maxpool operation on the rows
  # t.view(t.size(0),-1) = flatten into a 1D tensor each outcome for each input sample
  # passing from (n,a,b) to (n,a) dimensions
  def horizontal_pooling(t):
    return f.max_pool1d(t,t.size(2)).view(t.size(0),-1)
  
  def compute_batch_stats(self,model_output,batch):
    r_q,r_a = model_output    
    batch_loss = self.crit(r_q,r_a,batch.label.float()*2.0-1) # the loss wants -1|1 values, input values are 0|1
    sim = f.cosine_similarity(r_q,r_a)
    # compute the decision of the network
    pred = sim.clone()
    pred[sim > self.decision_threshold] = 1
    pred[sim <= self.decision_threshold] = 0
    correct_pred = (pred.squeeze() == batch.label.float()).sum().float()
    return sim,batch_loss,correct_pred

In [0]:
model = BaselineQACNN(10,400,3,5)
wiki_dataset = WikiQA_dataset(100,40)



---

*Draft snippets*

---

In [0]:
import numpy as np

#a = np.array([[[1,2,3],[4,5,6],[7,8,9]],[[10,11,12],[13,14,15],[16,17,18]]])
#a = np.array([[[[1],[2],[3]],[[4],[5],[6]],[[7],[8],[9]]],[[[10],[11],[12]],[[13],[14],[15]],[[16],[17],[18]]]])
a = np.array([[[[1,2,3],[4,5,6],[7,8,9]]],[[[10,11,12],[13,14,15],[16,17,18]]]])
t = torch.from_numpy(a).double()
#m = nn.MaxPool2d((1,3))
#po = f.max_pool1d(t,t.size(2))
#torch.cat(tuple(po),dim=1)
#po.view(t.size(0),-1)
c = nn.Conv2d(1,1,2).double()
c(t)


In [0]:
from torchvision import datasets,transforms
data = datasets.MNIST('.',train=True,download=True,transform= transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))]))

In [0]:
!ls